In [20]:
import pandas as pd
import time
import datetime
import numpy as np

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
df=pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end="+time.strftime("%Y%m%d"))[0]


In [23]:
df.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,"Dec 09, 2018",3473.23,3685.31,3469.09,3614.23,4947372847,62942160928
1,"Dec 08, 2018",3421.91,3506.04,3350.65,3476.11,5305024497,60531278392
2,"Dec 07, 2018",3512.59,3512.59,3280.23,3419.94,6835615448,59547645578
3,"Dec 06, 2018",3754.07,3874.97,3521.10,3521.10,5878333109,61303965508
4,"Dec 05, 2018",3958.89,3969.54,3753.99,3753.99,5302481574,65352496336


In [24]:
df=df.assign(Date=pd.to_datetime(df['Date']))

In [25]:
df.loc[df['Volume']=="-",'Volume']=0

In [26]:
df['Volume'] = df['Volume'].astype('int64')

In [27]:
df.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,2018-12-09,3473.23,3685.31,3469.09,3614.23,4947372847,62942160928
1,2018-12-08,3421.91,3506.04,3350.65,3476.11,5305024497,60531278392
2,2018-12-07,3512.59,3512.59,3280.23,3419.94,6835615448,59547645578
3,2018-12-06,3754.07,3874.97,3521.10,3521.10,5878333109,61303965508
4,2018-12-05,3958.89,3969.54,3753.99,3753.99,5302481574,65352496336


In [28]:
btc = { 'Day_dif': lambda df1: (df1['Close**']-df1['Open*'])/df1['Open*']}
df = df.assign(**btc)
df.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap,Day_dif
0,2018-12-09,3473.23,3685.31,3469.09,3614.23,4947372847,62942160928,0.040596
1,2018-12-08,3421.91,3506.04,3350.65,3476.11,5305024497,60531278392,0.015839
2,2018-12-07,3512.59,3512.59,3280.23,3419.94,6835615448,59547645578,-0.026377
3,2018-12-06,3754.07,3874.97,3521.10,3521.10,5878333109,61303965508,-0.062058
4,2018-12-05,3958.89,3969.54,3753.99,3753.99,5302481574,65352496336,-0.051757


In [29]:
for i in df: 
    kwargs = { 'Close_off_high': lambda x: 2*(x['High']- x['Close**'])/(x['High']-x['Low'])-1,
            'Volatility': lambda x: (x['High']- x['Low'])/(x['Open*'])}
    df = df.assign(**kwargs)

df.tail()

,Date,Open*,High,Low,Close**,Volume,Market Cap,Day_dif,Close_off_high,Volatility
2047,2013-05-02,116.38,125.60,92.28,105.21,0,1168517495,-0.095979,0.223890,0.286303
2048,2013-05-01,139.00,139.89,107.72,116.99,0,1298954594,-0.158345,0.423687,0.231439
2049,2013-04-30,144.00,146.93,134.05,139.00,0,1542813125,-0.034722,0.231366,0.089444
2050,2013-04-29,134.44,147.49,134.00,144.54,0,1603768865,0.075126,-0.562639,0.100342
2051,2013-04-28,135.30,135.98,132.10,134.21,0,1488566728,-0.008056,-0.087629,0.028677


In [30]:
df = df.dropna(how='any',axis=0) 

In [31]:
df.count()

Date              2051
Open*             2051
High              2051
Low               2051
Close**           2051
Volume            2051
Market Cap        2051
Day_dif           2051
Close_off_high    2051
Volatility        2051
dtype: int64

In [32]:
from sklearn import preprocessing
x=df[['Open*','High','Low','Volume','Market Cap','Day_dif','Close_off_high','Volatility']]
y=df[['Close**']]
z = x.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(z)
x = pd.DataFrame(x_scaled)
zz = y.values 
min_max_scaler = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler.fit_transform(zz)
y = pd.DataFrame(y_scaled)
                 



In [33]:
x.head()

,0,1,2,3,4,5,6,7
0,0.175436,0.180407,0.180001,0.207516,0.190848,0.417709,0.328739,0.122209
1,0.172791,0.171450,0.173737,0.222518,0.183446,0.379391,0.192612,0.087698
2,0.177464,0.171777,0.170013,0.286718,0.180426,0.314051,0.398735,0.130194
3,0.189906,0.189883,0.182752,0.246565,0.185818,0.258825,1.000000,0.187796
4,0.200460,0.194608,0.195068,0.222411,0.198248,0.274769,1.000000,0.106214


In [34]:
y.head()

,0
0,0.182501
1,0.175392
2,0.172501
3,0.177707
4,0.189694


In [37]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from keras.models import Sequential
from keras.layers import Activation, Dense,Flatten,LSTM,Dropout
kfold=KFold(10,True,7)
cvscores = []

In [39]:

x=np.array(x)
y=np.array(y)

for train, test in kfold.split(x,y):

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, 8))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, 8))

    output_size=1 
    neurons=100 
    activ_func="tanh"
    dropout=0.25
    loss="mean_squared_error" 
    optimizer="sgd"
    model = Sequential()
    model.add(LSTM(neurons,return_sequences=True,input_shape=(1,8)))
    model.add(LSTM(90))
 
    model.add(Dropout(dropout))
    
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer='adam',metrics =["mae"])
    model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=0,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


mean_absolute_error: 0.61%
mean_absolute_error: 0.51%
mean_absolute_error: 0.49%
mean_absolute_error: 0.87%
mean_absolute_error: 0.46%
mean_absolute_error: 0.52%
mean_absolute_error: 0.42%
mean_absolute_error: 0.56%
mean_absolute_error: 0.42%
mean_absolute_error: 0.51%
0.55% (+/- 0.12%)
